In [8]:
import numpy as np
import sys

import tensorflow as tf
from tensorflow.keras.layers import (Input, Layer, Dense, Lambda, 
                                     Dropout, Multiply, BatchNormalization, 
                                     Reshape, Concatenate, Conv2D, Permute)
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers.experimental.preprocessing import Resizing

from tensorflow.keras.datasets import cifar10

from datetime import datetime
import os

#Select GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
# IMPORTANT: SET RANDOM SEEDS FOR REPRODUCIBILITY
os.environ['PYTHONHASHSEED'] = str(420)
import random
random.seed(420)
np.random.seed(420)
tf.random.set_seed(420)

# Train Model to Be Explained

### Parameters

In [3]:
BATCH_SIZE = 32
EPOCHS = 100
LR = 1e-3
INPUT_SHAPE = (224, 224, 3)

### Load Data

In [25]:
from sklearn.model_selection import train_test_split

num_classes = 10

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, train_size=0.5, random_state=420)

x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')
#Resize to 224x224

print(x_train.shape[0], 'train samples')
print(x_val.shape[0], 'val samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# Make TF Dataset
ds_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
ds_val = tf.data.Dataset.from_tensor_slices((x_val, y_val))
ds_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))

50000 train samples
5000 val samples
5000 test samples


### Batch Data

In [26]:
def batch_data(dataset, fn, batch_size=32):
    dataset = dataset.map(fn)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    
    return dataset

### Reformat Data

In [27]:
def reformat(x, y):
    
    x = tf.cast(x, tf.float32)
    x = Resizing(INPUT_SHAPE[0], INPUT_SHAPE[1], interpolation='nearest')(x)
    x = tf.keras.applications.resnet50.preprocess_input(x)
    
    return (x, y)

ds_train = batch_data(ds_train, reformat, BATCH_SIZE)
ds_val = batch_data(ds_val, reformat, BATCH_SIZE)
ds_test = batch_data(ds_test, reformat, BATCH_SIZE)

### Model

In [29]:
from tensorflow.keras.applications.resnet50 import ResNet50

base_model = ResNet50(
    include_top=False, weights='imagenet', 
    input_shape=INPUT_SHAPE, pooling='avg'
)
base_model.trainable = True

model_input = Input(shape=INPUT_SHAPE, name='input')

net = base_model(model_input)
out = Dense(10, activation='softmax')(net)

model = Model(model_input, out)

# Metrics
METRICS = [ 
  tf.keras.metrics.AUC(name='auroc'),
  tf.keras.metrics.AUC(curve='PR', name='auprc'),
  tf.keras.metrics.TopKCategoricalAccuracy(k=1, name='accuracy'),
]

# Model Checkpointing
time = datetime.now().strftime("%Y%m%d_%H_%M_%S")
save_dir = 'model'
model_dir = os.path.join(os.getcwd(), save_dir, time)
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)
model_weights_path = os.path.join(model_dir, 'model_weights.h5')
checkpoint = ModelCheckpoint(model_weights_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only=True)

# LR Schedule
reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.95, patience=3, 
                             verbose=1, mode='min', cooldown=1, min_lr=1e-4)

# Early Stopping 
earlyStop = EarlyStopping(monitor="val_loss", mode="min", patience=10) 

# Compile Model
CALLBACKS = [checkpoint, earlyStop, reduceLR]
OPTIMIZER = tf.keras.optimizers.Adam(LR)

model.compile(
    loss='categorical_crossentropy',
    optimizer=OPTIMIZER,
    metrics=METRICS,
)

# Train Model
model.fit(ds_train,
          epochs = EPOCHS,
          validation_data = ds_val,
          callbacks = CALLBACKS)

# Get Checkpointed Model
print(model_weights_path)
model.load_weights(model_weights_path)
model.trainable = False

# Evaluate
model.evaluate(ds_test)

Epoch 1/100
1563/1563 [==============================] - ETA: 0s - loss: 0.7552 - auroc: 0.9664 - auprc: 0.8329 - accuracy: 0.7432
Epoch 00001: val_loss improved from inf to 0.88190, saving model to /gpfs/data/paulab/nj594/fast_shap/experiments/images/cifar10/model/20210514_14_39_21/model_weights.h5
1563/1563 [==============================] - 173s 111ms/step - loss: 0.7552 - auroc: 0.9664 - auprc: 0.8329 - accuracy: 0.7432 - val_loss: 0.8819 - val_auroc: 0.9553 - val_auprc: 0.8019 - val_accuracy: 0.7072
Epoch 2/100
1563/1563 [==============================] - ETA: 0s - loss: 0.4232 - auroc: 0.9886 - auprc: 0.9349 - accuracy: 0.8552
Epoch 00002: val_loss improved from 0.88190 to 0.61423, saving model to /gpfs/data/paulab/nj594/fast_shap/experiments/images/cifar10/model/20210514_14_39_21/model_weights.h5
1563/1563 [==============================] - 171s 110ms/step - loss: 0.4232 - auroc: 0.9886 - auprc: 0.9349 - accuracy: 0.8552 - val_loss: 0.6142 - val_auroc: 0.9758 - val_auprc: 0.8809

Epoch 20/100
1563/1563 [==============================] - ETA: 0s - loss: 0.0164 - auroc: 0.9998 - auprc: 0.9993 - accuracy: 0.9948
Epoch 00020: val_loss did not improve from 0.47743
1563/1563 [==============================] - 164s 105ms/step - loss: 0.0164 - auroc: 0.9998 - auprc: 0.9993 - accuracy: 0.9948 - val_loss: 0.6253 - val_auroc: 0.9736 - val_auprc: 0.9189 - val_accuracy: 0.8724
Epoch 21/100
1563/1563 [==============================] - ETA: 0s - loss: 0.0190 - auroc: 0.9999 - auprc: 0.9995 - accuracy: 0.9933
Epoch 00021: val_loss did not improve from 0.47743

Epoch 00021: ReduceLROnPlateau reducing learning rate to 0.0008145062311086804.
1563/1563 [==============================] - 164s 105ms/step - loss: 0.0190 - auroc: 0.9999 - auprc: 0.9995 - accuracy: 0.9933 - val_loss: 0.6210 - val_auroc: 0.9745 - val_auprc: 0.9212 - val_accuracy: 0.8794
Epoch 22/100
1563/1563 [==============================] - ETA: 0s - loss: 0.0172 - auroc: 0.9998 - auprc: 0.9993 - accuracy: 0.9946
Epo

[0.5485545992851257, 0.9772732853889465, 0.9263914823532104, 0.878600001335144]

# Save Images

In [30]:
#Get Directory 
save_dir = 'images'
images_dir = os.path.join(os.getcwd(), save_dir)
if not os.path.isdir(images_dir):
    os.makedirs(images_dir)
    
#unbatch
ds_test = ds_test.unbatch()
    
# Load 1,000 Images
processed_imgs = []
labels = []
for i, (x, y) in enumerate(ds_test):
    processed_imgs.append(x.numpy())
    labels.append(y.numpy())
    if i >= 999:
        break
        
# imgs = np.array(imgs)
processed_imgs = np.array(processed_imgs)
labels = np.array(labels)

# imgs.dump(os.path.join(images_dir, 'images.npy'))
processed_imgs.dump(os.path.join(images_dir, 'processed_images.npy'))
labels.dump(os.path.join(images_dir, 'labels.npy'))

#predictions
predictions = model.predict(processed_imgs)
predictions.dump(os.path.join(images_dir, 'predictions.npy'))